In [91]:
from importlib import reload
reload(io)
reload(info)
reload(base)

<module 'src.base' from '/home/ugur/python-projects/cefis2/src/base.py'>

In [95]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.auxiliary_functions as aux
import src.condition as con
from src.spread import group_by_spread
import src.base as base
from itertools import combinations
from src.spread import get_spread

## Veri Hazırlama

In [117]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [118]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [119]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [120]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [121]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [122]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">

In [123]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [124]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [125]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])